## Importing data and libraries

In [1]:
# 0
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
import re
from nltk.corpus import stopwords
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import networkx as nx
import matplotlib.pyplot as plt
from bokeh.models import Circle, MultiLine, Range1d, LabelSet
from bokeh.plotting import figure, from_networkx, show
from bokeh.models.sources import ColumnDataSource
from bokeh.palettes import Blues8
from bokeh.models.graphs import NodesAndLinkedEdges
from bokeh.io import save

# Loading the dataframe containing the comments and attribute labels
df = pd.read_excel("preprocessed_train.xlsx")
#df = df.head(200) # part for code testing
df["comment_text_lemm"] = df["comment_text_lemm"].astype(str)

# Splitting the dataset into training and testing sets
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)
display(train_df, test_df)

C:\Users\kripa\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kripa\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\kripa\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_clean,comment_text_nostop,comment_text_stem,comment_text_lemm
95,003b9f448ee4a29d,"""\n\nThanks. I can see that violating clearly ...",0,0,0,0,0,0,\n\nthanks i can see that violating clearly st...,thanks see violating clearly stated wikipedia ...,thank see violat clear state wikipedia polici ...,thank see violat clear state wikipedia polici ...
15,00078f8ce7eb276d,"""\n\nJuelz Santanas Age\n\nIn 2002, Juelz Sant...",0,0,0,0,0,0,\n\njuelz santanas age\n\nin juelz santana wa...,juelz santanas age juelz santana years old cam...,juelz santana age juelz santana year old came ...,juelz santana age juelz santana year old came ...
30,0011cc71398479c4,How could I post before the block expires? Th...,0,0,0,0,0,0,how could i post before the block expires the...,could post block expires funny thing think im ...,could post block expir funni thing think im un...,could post block expir funni thing think im un...
158,0061b075244dd234,"Once again, I responded to MileMoney's reasoni...",0,0,0,0,0,0,once again i responded to milemoneys reasoning...,responded milemoneys reasoning edit gave reaso...,respond milemoney reason edit gave reason edit...,respond milemoney reason edit gave reason edit...
128,0052a7e684beeb1a,""". (On Dec 14, 2006, a NIST scientist said """"....",0,0,0,0,0,0,on dec a nist scientist said the collapse o...,dec nist scientist said collapse towers magnit...,dec nist scientist said collaps tower magnitud...,dec nist scientist said collaps tower magnitud...
...,...,...,...,...,...,...,...,...,...,...,...,...
59,002264ea4d5f2887,Why can't you believe how fat Artie is? Did yo...,1,0,0,0,0,0,why cant you believe how fat artie is did you ...,cant believe fat artie see recent appearence t...,cant believ fat arti see recent appear tonight...,cant believ fat arti see recent appear tonight...
171,006a263a08b593c5,Note that the Sandbox is the right place to ex...,0,0,0,0,0,0,note that the sandbox is the right place to ex...,note sandbox right place experment thanks,note sandbox right place exper thank,note sandbox right place exper thank
131,0053978373606ba4,TCM \n\nI can find no evidence that acupressur...,0,0,0,0,0,0,tcm \n\ni can find no evidence that acupressur...,tcm find evidence acupressure tcm rather deriv...,tcm find evid acupressur tcm rather deriv tcm ...,tcm find evid acupressur tcm rather deriv tcm ...
17,000897889268bc93,REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski,0,0,0,0,0,0,redirect talkvoydan pop georgiev chernodrinski,redirect talkvoydan pop georgiev chernodrinski,redirect talkvoydan pop georgiev chernodrinski,redirect talkvoydan pop georgiev chernodrinski


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_clean,comment_text_nostop,comment_text_stem,comment_text_lemm
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,daww he matches this background colour im seem...,daww matches background colour im seemingly st...,daww match background colour im seem stuck tha...,daww match background colour im seem stuck tha...
14,00070ef96486d6f9,"Oh, and the girl above started her arguments w...",0,0,0,0,0,0,oh and the girl above started her arguments wi...,oh girl started arguments stuck nose doesnt be...,oh girl start argument stuck nose doesnt belon...,oh girl start argument stuck nose doesnt belon...
20,000b08c464718505,"""\n\n Regarding your recent edits \n\nOnce aga...",0,0,0,0,0,0,\n\n regarding your recent edits \n\nonce agai...,regarding recent edits please read wpfilmplot ...,regard recent edit pleas read wpfilmplot edit ...,regard recent edit plea read wpfilmplot edit f...
21,000bfd0867774845,"""\nGood to know. About me, yeah, I'm studying ...",0,0,0,0,0,0,\ngood to know about me yeah im studying nowde...,good know yeah im studying nowdeepu,good know yeah im studi nowdeepu,good know yeah im studi nowdeepu
37,0015f4aa35ebe9b5,pretty much everyone from warren county/surrou...,0,0,0,0,0,0,pretty much everyone from warren countysurroun...,pretty much everyone warren countysurrounding ...,pretti much everyon warren countysurround regi...,pretti much everyon warren countysurround regi...
48,001cadfd324f8087,"""\nAs for your claims of """"stalking"""", that is...",0,0,0,0,0,0,\nas for your claims of stalking that is absol...,claims stalking absolute rubbish serves aggrav...,claim stalk absolut rubbish serv aggrav situat...,claim stalk absolut rubbish serv aggrav situat...
50,001d8e7be417776a,"""\n\nBI, you said you wanted to talk\n\nAt the...",0,0,0,0,0,0,\n\nbi you said you wanted to talk\n\nat the b...,bi said wanted talk bottom lead section writte...,bi said want talk bottom lead section written ...,bi said want talk bottom lead section written ...
52,001e89eb3f0b0915,Are you threatening me for disputing neutralit...,0,0,0,0,0,0,are you threatening me for disputing neutralit...,threatening disputing neutrality know country ...,threaten disput neutral know countri quit comm...,threaten disput neutral know countri quit comm...
54,001ffdcc3e7fb49c,Awesome! Then I'll simply disregard your notic...,0,0,0,0,0,0,awesome then ill simply disregard your notice ...,awesome ill simply disregard notice thanks,awesom ill simpli disregard notic thank,awesom ill simpli disregard notic thank
57,00218d74784ce50b,"""\n\n GA Review II \n\nI'm sorry to say this, ...",0,0,0,0,0,0,\n\n ga review ii \n\nim sorry to say this but...,ga review ii im sorry say fail articles gan se...,ga review ii im sorri say fail articl gan seve...,ga review ii im sorri say fail articl gan seve...


## Tokenizing and labeling of data

In [2]:
# 1 Loading the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

labels = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
labels = torch.tensor(labels, dtype=torch.float32)

# Splitting the data into train and test sets
train_labels, test_labels = train_test_split(labels, test_size=0.2, random_state=42)

# Tokenizing the comment text using the BERT tokenizer
train_encodings = tokenizer(train_df["comment_text_lemm"].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df["comment_text_lemm"].tolist(), truncation=True, padding=True)

# Encoding the tokenized sequences using the BERT encoding scheme to obtain input features
train_input_ids = torch.tensor(train_encodings["input_ids"])
train_attention_mask = torch.tensor(train_encodings["attention_mask"])
test_input_ids = torch.tensor(test_encodings["input_ids"])
test_attention_mask = torch.tensor(test_encodings["attention_mask"])

# Making a TensorDataset for training and testing sets
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)


## Building BERT model and evaluation

In [3]:
# 2 Building a BERT-based NLP attention model with two layers ver2
class ToxicityClassifier(nn.Module):
    def __init__(self, num_labels):
        super(ToxicityClassifier, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

num_labels = 6  # Number of toxic attributes

In [ ]:
# 3 DataLoader for training and testing sets
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Making an instance of the ToxicityClassifier model
model = ToxicityClassifier(num_labels)

# Defining the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

# Training the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f"Epoch {epoch+1}/{5}: Average Loss = {avg_loss:.4f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/5: Average Loss = 0.4071


In [ ]:
# 4 Evaluating the model
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, _ = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        logits = model(input_ids, attention_mask)
        logits = torch.sigmoid(logits)
        predictions.extend(logits.cpu().numpy())

predictions = torch.tensor(predictions)


In [ ]:
# 5 Converting predictions to binary values (0 or 1) based on a threshold
threshold = 0.5
binary_predictions = (predictions >= threshold).to(torch.float32)

# Convert labels to numpy.ndarray if they are torch.Tensor
if isinstance(labels, torch.Tensor):
    labels = labels.cpu().numpy()

# Ensure that predictions and labels have the same number of samples
if len(binary_predictions) != len(labels):
    raise ValueError("Number of samples in predictions and labels does not match.")

# Check if there are any positive samples in the labels
if np.sum(labels) == 0:
    print("No positive samples in the labels.")
else:
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, binary_predictions)
    precision = precision_score(labels, binary_predictions, average="micro", zero_division=1)
    recall = recall_score(labels, binary_predictions, average="micro", zero_division=1)
    f1 = f1_score(labels, binary_predictions, average="micro", zero_division=1)

    # Print the evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

## Finding toxic words for network

In [ ]:
# 6 Fixing data for futher analysis
init_notebook_mode(connected=True)

train = df
train['comment_text'] = train['comment_text'].astype(str)
train

In [ ]:
# 7 Reindexing
train.index = train['id']
x_train = train['comment_text']
y_train = train.iloc[:, 2:]

In [ ]:
# 7.1 If some kind of toxicity is detected, the sum across rows will yield one, 
# and the subtraction will give zero, and one otherwise
y_train['clean'] = 1 - y_train.sum(axis=1) >= 1  

In [ ]:
# 7.2 The sum operation yield a series, and a series behaves like a dictionary
# as it has the items function that returns index-value tuples.
kinds, counts = zip(*y_train.sum(axis=0).items())

In [ ]:
# 7.3
bars = go.Bar(
        y=counts,
        x=kinds,
    )

layout = go.Layout(
    title="Class distribution in train set"
)

fig = go.Figure(data=[bars], layout=layout)
iplot(fig, filename='bar')

In [ ]:
# 8
nlp = spacy.load("en_core_web_sm", disable=['parser', 'tagger', 'ner'])
stops = stopwords.words("english")

In [ ]:
# 8.1 Additional normalization of data
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [ ]:
# 8.2
x_train_lemmatized = x_train.apply(normalize, lowercase=True, remove_stopwords=True)

In [ ]:
# 8.3 Demo
x_train_lemmatized.sample(1).iloc[0]

In [ ]:
# 9 Counting toxic words, that are met most often in comments
from collections import Counter
word_counts = dict()

for kind in y_train.columns:
    word_counts[kind] = Counter()
    comments = x_train_lemmatized[y_train[kind]==1]
    for _, comment in comments.iteritems():
        word_counts[kind].update(comment.split(" "))
def most_common_words(kind, num_words=15):
    words, counts = zip(*word_counts[kind].most_common(num_words)[::-1])
    bars = go.Bar(
        y=words,
        x=counts,
        orientation="h"
    )

    layout = go.Layout(
        title="Most common words of the class \"{}\"".format(kind),
        yaxis=dict(
            ticklen=8  # to add some space between yaxis labels and the plot
        )
    )

    fig = go.Figure(data=[bars], layout=layout)
    iplot(fig, filename='bar')
most_common_words("toxic")

In [ ]:
# 9.1 Plot for every type of comments, showing words
most_common_words("severe_toxic")

In [ ]:
# 9.2
most_common_words("threat")

In [ ]:
# 9.3
most_common_words("clean")

In [ ]:
# 10 Converting the 'comment_text' column to string
df['comment_text_lemm'] = df['comment_text_lemm'].astype(str)

# Concatenating all the toxic comments into a single string
all_toxic_comments = ' '.join(df[df['toxic'] == 1]['comment_text_lemm'])

# Splitting the string into individual words
all_toxic_words = all_toxic_comments.split()

# Calculation of the frequency of each word
word_frequency = pd.Series(all_toxic_words).value_counts()

## Network visualization

In [ ]:
# 11 Getting the word embeddings for toxic words
toxic_words = all_toxic_words

# Loading the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Tokenizing the toxic words
tokenized_words = tokenizer(toxic_words, padding=True, truncation=True, return_tensors='pt')

# Forwarding pass to obtain word embeddings
with torch.no_grad():
    outputs = model(**tokenized_words)
    word_embeddings = outputs.last_hidden_state.squeeze(0)

# Reshaping the word embeddings to remove the batch dimension
word_embeddings = word_embeddings.reshape(word_embeddings.size(0), -1)

# Calculating similarity scores between word embeddings
similarity_scores = cosine_similarity(word_embeddings.cpu().numpy())

# Creating a graph
G = nx.Graph()

# Adding nodes to the graph with word signatures
for i, word in enumerate(toxic_words):
    signature = tokenizer.convert_ids_to_tokens(tokenized_words['input_ids'][i].tolist())
    G.add_node(word, signature=signature)

# Adding edges to the graph
for i, word1 in enumerate(toxic_words):
    for j, word2 in enumerate(toxic_words):
        if i != j:
            similarity = similarity_scores[i, j]
            G.add_edge(word1, word2, weight=similarity)

# Drawing the graph
pos = nx.spring_layout(G, seed=42)
weights = [data['weight'] for _, _, data in G.edges(data=True)]
edges = nx.draw_networkx_edges(G, pos, edge_color=weights, edge_cmap=plt.cm.Reds, width=2)
nodes = nx.draw_networkx_nodes(G, pos, node_color='blue', alpha=0.7)

# Adding word signatures to the node labels
node_labels = {node: ' '.join(G.nodes[node]['signature']) for node in G.nodes}
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=6)

# Adding colorbar for edge weights
cbar = plt.colorbar(edges)
cbar.set_label('Connection Strength')

plt.title('Toxic Words Network')
plt.figure(figsize=(12, 8)) 
plt.axis('off')
plt.show()

First version of network is not well-representing data. So all the additional info will be controlled by making interactive plot.

In [ ]:
from transformers import BertTokenizer
from bokeh.models import Circle, MultiLine, Range1d
from bokeh.plotting import figure, from_networkx, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import LabelSet
from bokeh.palettes import Blues8
from bokeh.models.graphs import NodesAndLinkedEdges
from bokeh.io import save
import networkx as nx

# 12 Choosing colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

# Choosing attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

# Picking a color palette
color_palette = Blues8

title = 'Toxic Words Network'

# Establishing which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
    ("Word:", "@signature"),
    #("Centrality Degree:", "@degree"),
]

# Creating a plot — seting dimensions, toolbar, and title
plot = figure(tooltips=HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
              plot_width=1080, plot_height=900,  # Adjust the plot width and height
              x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title=title)

# Applying force-directed layout (spring layout) to the graph
pos = nx.spring_layout(G)

# Calculating node degrees
node_degrees = dict(G.degree())

# Setting node attributes
network_graph = from_networkx(G, pos, scale=2, center=(0, 0))
network_graph.node_renderer.glyph = Circle(size=15, fill_color="#87C38F", fill_alpha=0.7)
network_graph.node_renderer.selection_glyph = Circle(size=15, fill_color="#17BEBB", fill_alpha=0.7)
network_graph.node_renderer.hover_glyph = Circle(size=15, fill_color="#C44536", fill_alpha=0.7)

# Setting edge attributes
network_graph.edge_renderer.glyph = MultiLine(line_color="black", line_width=1.5)
network_graph.edge_renderer.selection_glyph = MultiLine(line_color="black", line_width=1.5)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color="#C44536", line_width=2.5)

# Enabling highlighting of connected nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

# Adding the graph to the plot
plot.renderers.append(network_graph)

# Creating a BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Adding word signatures and centrality degrees as labels
node_labels = {}
for node in G.nodes:
    signature = G.nodes[node]['signature']
    degree = node_degrees[node]
    if len(signature) > 1:
        node_name = tokenizer.tokenize(node)
        node_name = [token for token in node_name if not tokenizer.convert_tokens_to_string([token]) in tokenizer.all_special_tokens]
        node_labels[node] = f"{node_name[1:-1]}\nDegree: {degree}"
    else:
        node_labels[node] = f"Degree: {degree}"

x = [pos[node][0] for node in G.nodes]
y = [pos[node][1] for node in G.nodes]

source = ColumnDataSource({'x': x, 'y': y, 'word': list(node_labels.values()), 'signature': list(node_labels.values()), 'degree': list(node_degrees.values())})
labels = LabelSet(x='x', y='y', text='word', text_font_size='6pt', source=source, render_mode='canvas')

# Showing the plot
plot.add_layout(labels)
show(plot)
save(plot, filename=f"{title}-1.html")